In [1]:
pip install ftfy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install Unidecode

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install SpaCy

Note: you may need to restart the kernel to use updated packages.


In [15]:
import re
from ftfy import fix_text
import spacy
import json
from unidecode import unidecode
spacy_model = spacy.load('en_core_web_sm')

# open write and read json file
with open('gg2013.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Store RT totals from text
rt_totals = {}
for data_text in data:
    text = data_text['text']

    # The following cleaning must be done to combine tweets properly
    # Remove URLS
    text= re.sub(r'https?://\S+|www\.\S+', '', text)
    #Fix encoding issues
    text = fix_text(text)
    #Standardize special characters / emojis to Unicode
    text = unidecode(text)

    #extract RT data from text
    retweets = re.findall(r'\b(?:(?:RT )|\")@(\w+):? (.*?)(?=(?:\"\Z)|\Z|(?:\b RT @\w+:))', text)
    
    # Update retweet totals
    for rt in retweets:
        if rt not in rt_totals:
            rt_totals[rt] = 1
        else:
            rt_totals[rt] += 1

# Combine retweets with existing tweets
for data_text in data:
    original_text = data_text['text']

    # Find number of retweets
    tweet_key = (data_text['user']['screen_name'], original_text)
    data_text['retweets'] = rt_totals.pop(tweet_key, 0)

# Add retweeted tweets not in original dataset
for tweet_key in list(rt_totals):
    tweet = {'text': tweet_key[1], \
             'user': {'screen_name': tweet_key[0]}, \
             'retweets': rt_totals[tweet_key]}
    data.append(tweet)

preprocessed_data = []
user_metadata = {}
# loop to fix text{} in json data
for data_text in data:
    original_text = data_text['text']
    
    # Update user weights based on retweet value
    user_name = data_text['user']['screen_name']
    if user_name not in user_metadata:
        user_metadata[user_name] = {'num_tweets': 0, 'rt_total': 0, 'rt_average': 0}
    user_metadata[user_name]['num_tweets'] += 1
    user_metadata[user_name]['rt_total'] += data_text['retweets']
    user_metadata[user_name]['rt_average'] = user_metadata[user_name]['rt_total'] \
        / user_metadata[user_name]['num_tweets']

    # Remove retweets
    remove_retweets = re.sub(r'(?:\A| )(?:(?:RT )|\")@\w+:?.*', '', original_text)
    #clean white space
    white_space = " ".join(remove_retweets.split())
    #keep tabs/newline characters
    tab_newline = re.sub(' +', ' ', white_space)
    # lowcase letters
    low_letter = tab_newline.lower()
    #remove hashtags
    hash_tag = re.sub(r'#', '', low_letter)
    # Remove empty tweets
    if hash_tag == '':
        continue
    #After pre-processing, information going back to orginal json data
    data_text['text'] = hash_tag
    preprocessed_data.append(data_text)

# Save data
with open('gg2013_user_metadata.json', 'w', encoding='utf-8') as output_file:
    json.dump(user_metadata, output_file, indent=6)
with open('gg2013_preprocessed.json', 'w', encoding='utf-8') as output_file:
    json.dump(preprocessed_data, output_file, indent=6)



###Later use
    #spacy model to syntactic parsing
    # spacy_output = spacy_model(low_letter)
    # for token in spacy_output:
    #     print(
    #     token.text,
    #     token.dep_,
    #     token.head.text,
    #     token.head.pos_,
    #     [child for child in token.children]
    #     )
